### IMPORT REQUIREMENTS

In [1]:
import pandas as pd
import numpy as np
import ta

In [2]:
XAU = pd.read_csv('./XAU_USD Historical Data.csv')
BTC = pd.read_csv('./Bitcoin Historical Data.csv')

### DATA PREPROCESSING

In [3]:
def df_preprocess(df):

    # change 'Date' to datetime format & sort by 'Date'
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(by='Date', inplace=True)

    # change data type to float
    df['High'] = df['High'].str.replace(',', '').astype(float)
    df['Low'] = df['Low'].str.replace(',', '').astype(float)
    df['Open'] = df['Open'].str.replace(',', '').astype(float)
    df['Price'] = df['Price'].str.replace(',', '').astype(float)
    
    df['Change %'] = df['Change %'].str.rstrip('%').astype('float') / 100.0

    df.replace(np.nan, '12345.12345', inplace=True)
    df[df['Vol.'] == '12345.12345']
    df['Volume'] = df['Vol.'].str[:-1].astype(float)
    df['m'] = df['Vol.'].str[-1]
    change_value = {'K': 1000, 'M': 1000000, 'B': 1000000000, '5': 1}
    df['m'] = df['m'].map(change_value)
    df['Vol.'] = df['Volume']*df['m']
    df.replace(12345.1234, np.nan, inplace=True)
    df.drop(['Volume', 'm'], axis=1, inplace=True)

    return df

XAU = df_preprocess(XAU)
BTC = df_preprocess(BTC)

In [4]:
XAU_backup = XAU.copy()
BTC_backup = BTC.copy()

* Technical Feature Extraction

In [11]:
df = XAU

df['ROC_26'] = ta.momentum.ROCIndicator(close=df['Price'], window=24).roc()
df['ROC-34'] = ta.momentum.ROCIndicator(close=df['Price'], window=34).roc()

df['EMA_14'] = ta.trend.EMAIndicator(close=df['Price'], window=14).ema_indicator()
df['EMA_50'] = ta.trend.EMAIndicator(close=df['Price'], window=50).ema_indicator()
df['EMA_120'] = ta.trend.EMAIndicator(close=df['Price'], window=120).ema_indicator()

df['RSI_28'] = ta.momentum.RSIIndicator(close=df['Price'], window=28).rsi()
df['RSI_28_SMA14'] = ta.trend.SMAIndicator(close=df['RSI_28'], window=14).sma_indicator()

df['MACD_long_macd'] = ta.trend.MACD(close=df['Price'], window_slow=120, window_fast=52, window_sign=20).macd()
df['MACD_long_diff'] = ta.trend.MACD(close=df['Price'], window_slow=120, window_fast=52, window_sign=20).macd_diff()
df['MACD_long_signal'] = ta.trend.MACD(close=df['Price'], window_slow=120, window_fast=52, window_sign=20).macd_signal()

df['MACD_mid_macd'] = ta.trend.MACD(close=df['Price'], window_slow=52, window_fast=26, window_sign=10).macd()
df['MACD_mid_diff'] = ta.trend.MACD(close=df['Price'], window_slow=52, window_fast=26, window_sign=10).macd_diff()
df['MACD_mid_signal'] = ta.trend.MACD(close=df['Price'], window_slow=52, window_fast=26, window_sign=10).macd_signal()

df['BB_50_high'] = ta.volatility.BollingerBands(close=df['Price'], window=50, window_dev=2).bollinger_hband()
df['BB_50_ligh'] = ta.volatility.BollingerBands(close=df['Price'], window=50, window_dev=2).bollinger_lband()
df['BB_50_width'] = ta.volatility.BollingerBands(close=df['Price'], window=50, window_dev=2).bollinger_wband()
df['BB_50_percentage'] = ta.volatility.BollingerBands(close=df['Price'], window=50, window_dev=2).bollinger_pband()

df['IchiCloud_SpanA'] = ta.trend.IchimokuIndicator(high=df['High'], low=df['Low'], visual=True).ichimoku_a()
df['IchiCloud_SpanB'] = ta.trend.IchimokuIndicator(high=df['High'], low=df['Low'], visual=True).ichimoku_b()
df['IchiCloud_KijunSen'] = ta.trend.IchimokuIndicator(high=df['High'], low=df['Low'], visual=True).ichimoku_base_line()
df['IchiCloud_TenkanSen'] = ta.trend.IchimokuIndicator(high=df['High'], low=df['Low'], visual=True).ichimoku_conversion_line()

In [5]:
df = XAU.copy()

In [30]:
ta.trend.IchimokuIndicator(high=df['High'], low=df['Low'], visual=True).ichimoku_conversion_line()

3888         NaN
3887         NaN
3886         NaN
3885         NaN
3884         NaN
          ...   
4       2008.645
3       2026.340
2       2061.395
1       2062.455
0       2063.650
Name: ichimoku_conv_9_26, Length: 3889, dtype: float64

In [20]:
df['MACD_long_macd'] = ta.trend.MACD(close=df['Price'], window_slow=120, window_fast=52, window_sign=20).macd()
df['MACD_long_diff'] = ta.trend.MACD(close=df['Price'], window_slow=120, window_fast=52, window_sign=20).macd_diff()
df['MACD_long_signal'] = ta.trend.MACD(close=df['Price'], window_slow=120, window_fast=52, window_sign=20).macd_signal()
df

,Date,Price,Open,High,Low,Vol.,Change %,EMA_14,EMA_50,EMA_120,RSI_28,RSI_28_SMA14,MACD_long_macd,MACD_long_diff,MACD_long_signal
3888,2009-01-01,882.15,881.30,882.15,881.30,NaN,0.0022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3887,2009-01-02,876.55,882.80,888.35,868.10,NaN,-0.0063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3886,2009-01-05,859.00,878.00,884.55,844.50,NaN,-0.0200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3885,2009-01-06,864.15,859.85,870.70,839.55,NaN,0.0060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3884,2009-01-07,841.95,864.60,867.35,835.80,NaN,-0.0257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2023-11-30,2035.75,2044.80,2047.59,2031.84,NaN,-0.0043,2003.609515,1963.874691,1942.872350,62.657595,58.111526,19.847779,8.695301,11.152478
3,2023-12-01,2070.90,2034.49,2075.34,2033.75,NaN,0.0173,2012.581580,1968.071762,1944.988509,66.635653,59.188234,21.813878,9.646029,12.167849
2,2023-12-04,2029.74,2071.25,2135.90,2020.34,NaN,-0.0199,2014.869369,1970.490124,1946.389360,59.002817,59.535228,22.788031,9.608736,13.179295
1,2023-12-05,2019.42,2030.15,2041.33,2010.02,NaN,-0.0051,2015.476120,1972.408943,1947.596478,57.296324,59.805429,23.476861,9.316845,14.160016
